# import package

In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils import data
import random

# mock data

In [2]:
def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise.

    Defined in :numref:`sec_utils`"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, torch.reshape(y, (-1, 1))

In [3]:
true_w = torch.tensor([2, -3.4])
true_b = 5.2
features, labels = synthetic_data(true_w, true_b, 1000)

# Read Data

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [5]:
batch_size = 5
data_iter = load_array((features, labels), batch_size)

In [6]:
next(iter(data_iter))

[tensor([[-0.0089,  1.7604],
         [ 0.4471, -1.1932],
         [ 1.9845, -0.0067],
         [ 0.3872,  0.4186],
         [-1.2348,  0.2109]]),
 tensor([[-0.8077],
         [10.1472],
         [ 9.1750],
         [ 4.5577],
         [ 1.9980]])]

# define model

In [7]:
net = nn.Sequential(nn.Linear(2, 1))

## init params

In [8]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## define loss

In [9]:
loss = nn.MSELoss()

## define optimizer

In [10]:
learning_rate = 0.03
trainer = torch.optim.SGD(net.parameters(), lr=learning_rate)

# train model

In [13]:
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000101
epoch 2, loss 0.000102
epoch 3, loss 0.000101
epoch 4, loss 0.000101
epoch 5, loss 0.000101
epoch 6, loss 0.000102
epoch 7, loss 0.000102
epoch 8, loss 0.000101
epoch 9, loss 0.000103
epoch 10, loss 0.000102


In [14]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)


w的估计误差： tensor([ 9.5582e-04, -3.0518e-05])
b的估计误差： tensor([-0.0009])
